# Решение задачи "Анализ ЭКГ-сигналов для диагностики сердечных патологий" для конкурса AI challenge

Оглавление
- Исследование области задачи
- Обработка и анализ данных
- Тестирование различных моделей

## Исследование области задачи
- SCP-ECG - http://masters.donntu.ru/2008/kita/golovach/library/4_ref/pub.html
- стадии инфаркта

## Обработка и анализ данных

Локальные переменные

In [ ]:
import os
os.environ['DATA_DIR'] = './data' # ты указываешь путь к своей папке

Используемые модули

In [ ]:
# Для данных
import pandas as pd
import numpy as np
import json
import os

# Для плюшек 
import sklearn as sk

# Для красоты
import seaborn as sns
import matplotlib.pyplot as plt
from pprint import pprint
import typing

Полезные функции

In [ ]:
def get_hr(folder: str, hr_num: str) -> np.array:
    with open(f'{os.environ["DATA_DIR"]}/{folder}/{hr_num}.npy', "rb") as f:
        return np.load(f, allow_pickle=True)


def flatten_list(lst: typing.List[any]) -> typing.List[any]:
    new_lst = []
    for elem in lst:
        if isinstance(elem, list):
            new_lst.extend(flatten_list(elem))
        else:
            new_lst.append(elem)
            
    return new_lst

def global_info(meta: pd.DataFrame):
    for column in meta.columns:
        print(f'Column name: {column} {round(meta[column].notna().sum() / len(meta) * 100, 2)}%')
        print(meta[column].value_counts() if len(meta[column].unique()) < 14 else f'so much unique values\n{meta[column].describe()}')
        print()

Пример использования

In [ ]:
a = get_hr(folder='train', hr_num='15857_hr')
sns.lineplot(data=a.flatten()[:1500])

ecg_columns = ['report', 'scp_codes', 'heart_axis', 'infarction_stadium1', 'infarction_stadium2', 'validated_by', 
'second_opinion', 'initial_autogenerated_report', 'validated_by_human', 'baseline drift', 'static_noise', 'burst_noise', 'electrodes_problems', 
'extra_beats']

Загрузка данных

In [ ]:
meta = pd.read_csv(f'{os.environ["DATA_DIR"]}/train/train_meta.csv')
diagnosis = pd.read_csv(f'{os.environ["DATA_DIR"]}/train/train_gts.csv')

In [ ]:
meta['age'].describe()

In [ ]:
meta.head(3)

In [ ]:
diagnosis.head(3)

In [ ]:
diagnosis['myocard'].value_counts()

Просмотрим информацию о нашем датасете </br>
https://physionet.org/content/ptb-xl/1.0.3/ - еще здесь нада

In [ ]:
meta.info()

Удалим те данные, которые не несут важной для нас информации

In [ ]:
useless_columns = ['ecg_id', 'patient_id', 'nurse', 'site', 'device', 'recording_date', 'filename_lr', 'filename_hr', 'report']
meta.drop(columns=useless_columns, inplace=True)

Создадим список тех столбцов, которые возможно не несут важной информации (мы проверим это при обучении)

In [ ]:
strange_columns = ['age', 'sex', 'pacemaker', 'group']

Рассмотрим те столбцы, в которых есть много пропусков

In [ ]:
global_info(meta)

На основе этих данных выделим список столбцов с множество недостающих значений

In [ ]:
empty_columns = ['height', 'weight', 'heart_axis']
meta.drop(columns=empty_columns, inplace=True)

Удалим те строки, в которых electrodes_problem

In [ ]:
meta.drop(meta[meta['electrodes_problems'].notna()].index, inplace=True)
meta.drop(columns=['electrodes_problems'], inplace=True)
meta.drop(1514, inplace=True) # Хех пока

In [ ]:
meta = meta.reset_index(drop=True)

#### age
- определение: возраст
- диапозон от 3 до 300.
#### sex
- определение: пол
- мужской(1), женский(0).
#### report
- определение: строка отчета, сгенерированная кардиологом или автоматически интерпретируемая ЭКГ-устройством, которая была преобразована в стандартизированный набор выписок SCP-ЭКГ (scp_codes)
- диапозон безумно большой.
#### scp_codes
- определение: SCP-показания ЭКГ
- в виде словаря с записями вида statement: likelihoodгде вероятность установлена равной 0, если неизвестно, диапозон безумно большой.
#### infarction_stadium1
- определение: стадия инфаркта1
- диапозон['Stadium I', 'Stadium I-II', 'Stadium II', 'Stadium II-III', 'Stadium III'].
#### infarction_stadium2
- определение: стадия инфаркта2
- диапозон['Stadium I', 'Stadium II', 'Stadium III'].
#### validated_by
- определение: в переводчике пишет "подтверждено" 
- диапозон от 0 до 9.
#### second_opinion
- определение: второе мнение(консультация второго специалиста)
- диапозон состоит из True и False.
#### initial_autogenerated_report
- определение: первоначальный автоматически сгенерированный отчет
- диапозон состоит из True и False. 
#### validated_by_human
- определение: подтверждено человеком
- диапозон состоит из True и False. 
#### baseline_drift
- определение: смещение базовой линии (на самой экг карточки(npy))
- диапозон *noise_types*
#### static_noise
- определение: статический шум
- диапозон *noise_types*
#### burst_noise
- определение: Шум взрыва — тип электронного шума, который возникает в полупроводниках и ультратонких оксидных пленках затвора
- диапозон большой
#### extra_beats
- определение: дополнительные удары, предоставляемые для подсчета дополнительных систол и кардиостимулятор для получения паттернов сигналов, указывающих на активный кардиостимулятор
- диапозон['1,V1,V2', '1,V2', '1ES', '1VES', '2,V1', '2,V3', '2,V4', '2ES', '2ES,SVES', '3,alles', '3ES', '4ES', '4VES', '5,alles', 'ES', 'SVES', 'SVES1,V5', 'SVES1,alles', 'VES', 'VES,SVES', 'VES,SVES1,alles', 'VES,SVES3,alles', 'VES1,II-AVF', 'VES1,II-V6', 'VES1,alles', 'VES2,alles', 'VES3,alles', 'VES4,alles', 'VES6,alles'].
#### pacemaker
- определение: кардиостимулятор
- диапозон['PACE????, nan', 'ja, pacemaker'].
#### strat_fold
- определение: свертки перекрестной проверки: рекомендуемые 10-кратные разбиения для поездных тестов, полученный с помощью стратифицированной выборки с учетом распределения пациентов, т. е. все записи конкретного пациента были отнесены к одному и тому же сгибу
- Записи в fold 9 и 10 прошли по крайней мере одну оценку человеком и поэтому имеют особенно высокое качество маркировки. Поэтому мы предлагаем использовать fold 1-8 в качестве обучающего набора, fold 9 в качестве набора для проверки и fold 10 в качестве набора для тестирования, диапозон от 1 до 10
#### record_name
- определение: имя записи
- состоит из 5 цифр и после цифр идет '_hr'
#### group
- определение: группа
- диапозон от 1 до 3

In [ ]:
meta.info()

Обработаем оствашиеся не числовые столбцы (кроме report и record_name и категориальных)
- baseline_drift
- static_noise
- scp_codes

In [ ]:
noise_types = ['I', 'II', 'III', 'AVR', 'AVL', 'AVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']

def str_to_noise(s: str) -> typing.List[str]:
    if not isinstance(s, str):
        return []
    noises = []
    
    if 'alles' in s: # alles - all
        return noise_types
    
    vnable = False
    for elem in s.split(','):
        elem = elem.strip(',- ').upper()
        if not elem:
            continue
        if 'V' in elem and 'A' not in elem:
            vnable = True
        else:
            if vnable and elem in map(str, range(1, 7)):
                elem = 'V' + elem
        if '-' in elem:
            elem = elem.split('-')
            if 'V' in elem[0] and 'V' not in elem[1]:
                elem[1] = 'V' + elem[1]
            noises.extend(noise_types[noise_types.index(elem[0].strip(',- ')):noise_types.index(elem[1].strip(',- '))+1])
        else:
            noises.append(elem)
            
    return noises

def scp_convert(s: str) -> typing.Dict[str, int]:
    return json.loads(s.replace("'", '"'))

Превращение в список

In [ ]:
meta['baseline_drift'] = meta['baseline_drift'].apply(str_to_noise)
meta['static_noise'] = meta['static_noise'].apply(str_to_noise)
params = set(flatten_list([list(eval(codes).keys()) for codes in meta['scp_codes'].unique().tolist()]))
meta['scp_codes'] = meta['scp_codes'].apply(scp_convert)

static_noise

In [ ]:
for elem in noise_types:
    meta[f'static_noise_{elem}'] = False
    for id, row in meta.iterrows():
        meta.loc[id, f'static_noise_{elem}'] = bool(elem in row['static_noise'])

baseline_drift

In [ ]:
for elem in noise_types:
    meta[f'baseline_drift_{elem}'] = False
    for id, row in meta.iterrows():
        meta.loc[id, f'baseline_drift_{elem}'] = bool(elem in row['baseline_drift'])

scp_codes

In [ ]:
for elem in params:
    meta[f'scp_{elem}'] = meta['scp_codes'].apply(lambda x: x.get(elem))

Пока пока

In [ ]:
meta.drop(columns=['baseline_drift', 'static_noise', 'scp_codes'], inplace=True)

Заменим строки в названиях, так как они малину портят

In [ ]:
str_columns = ['infarction_stadium1', 'infarction_stadium2', 'burst_noise', 'pacemaker', 'extra_beats']

meta = pd.get_dummies(meta, columns=str_columns, dtype=bool)

In [ ]:
global_info(meta)

Добавление столбцов с ЭКГ записями

In [ ]:
hrs = pd.DataFrame([get_hr('train', hr_name).flatten() for hr_name in meta['record_name']])

In [ ]:
meta = pd.concat([meta, hrs], axis=1)

In [ ]:
meta.info()

Заменим оставшиеся Nan

In [ ]:
for col in meta.columns:
    if meta[col].isna().sum():
        meta[col] = meta[col].fillna(0)

Разбеиние на обучающую и тестовую выборки

In [ ]:
test_meta = meta.loc[meta['strat_fold'] > 8].drop(columns=['strat_fold']).reset_index(drop=True)
train_meta = meta.loc[meta['strat_fold'] < 9].drop(columns=['strat_fold']).reset_index(drop=True)

In [ ]:
train_diagnosis = diagnosis.loc[diagnosis['record_name'].isin(train_meta['record_name'])].reset_index(drop=True)
test_diagnosis = diagnosis.loc[diagnosis['record_name'].isin(test_meta['record_name'])].reset_index(drop=True)

In [ ]:
train_meta.info()

In [ ]:
train_diagnosis.info()

In [ ]:
test_meta.info()

In [ ]:
test_diagnosis.info()

In [ ]:
test = test_meta.merge(test_diagnosis, how="outer", on=["record_name"]).drop(columns=["record_name"])
train = train_meta.merge(train_diagnosis, how="outer", on=["record_name"]).drop(columns=["record_name"])

In [ ]:
train_X = train.drop(columns=['myocard'])
train_Y = train['myocard']

In [ ]:
test_X = test.drop(columns=['myocard'])
test_Y = test['myocard']

## Тестирование различных моделей

Превращение в np (для некотрых надо)

In [ ]:
train_X_np = train_X.to_numpy()
train_Y_np = train_Y.to_numpy()
test_X_np = test_X.to_numpy()
test_Y_np = test_Y.to_numpy()

Используемые модули (не точно)

In [ ]:
# Нейронки (какие-то уберем, разобраться сначала, какие для этого подходят)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Метрики
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

### KNN

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)
knn = knn.fit(train_X_np, train_Y_np)

knn_train_preds = knn.predict(train_X_np)
knn_train_preds_proba = knn.predict_proba(train_X_np)[:, 1]

knn_test_preds = knn.predict(test_X_np)
knn_test_preds_proba = knn.predict_proba(test_X_np)[:, 1]

In [ ]:
print('On train: ')
print(f'точность - {accuracy_score(train_Y_np, knn_train_preds)}')
print(f'плошадь под roc-кривой - {roc_auc_score(train_Y_np, knn_train_preds_proba)}')
print(f'оценка f1 - {f1_score(train_Y_np, knn_train_preds)}') # т.к. пациентов с инфарктом и без него разное количество, это важная метрика

print('On test: ')
print(f'точность - {accuracy_score(test_Y_np, knn_test_preds)}')
print(f'плошадь под roc-кривой - {roc_auc_score(test_Y_np, knn_test_preds_proba)}')
print(f'оценка f1 - {f1_score(test_Y_np, knn_test_preds)}')

In [ ]:
# TODO далее все также как в бейзлайне/(ноутбуках, которые я скидывал), также стоит почитать доки у sk и возможно некоторые параметры при создании модельки подредачить
# TODO после прогонки на дате, попробовать откинуть один из strange_columns, попробовать еще раз
# TODO если с дефолтными модельками закончим, то можно будет переходить к тому, что в импортах отмечено кок посложнее

# И конечно же все записывать, для этого можно просто табличку создать

А пока часть посложнее

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader

Загрузка датасетов в dataloaderы

In [ ]:
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

Создаем модель

In [ ]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

Обучаем и тестируем

In [ ]:
# https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html